In [ ]:
import pandas as pd
import numpy as np

In [ ]:
open_moji_df = pd.read_csv('../data/openmoji.csv')
emojipedia_df = pd.read_csv('../data/emojipedia/emojis_details.csv')
emojipedia_df_with_grouping=pd.read_csv('../data/emojipedia/emojis_with_subgroups.csv')